## 1. Datenimport und Initialisierung

In diesem Abschnitt werden die OpenJur-Urteilstexte aus dem Datenverzeichnis eingelesen und die technische Datenbasis für die nachfolgenden Verarbeitungsschritte geschaffen. Dazu werden die benötigten Bibliotheken importiert und die verfügbaren Textdateien identifiziert.


### 1.1 Import der benötigten Bibliotheken

Zu Beginn werden die für die weitere Verarbeitung erforderlichen Python-Bibliotheken importiert. Diese umfassen Funktionen für Dateizugriffe, reguläre Ausdrücke, Datenverarbeitung mit Pandas sowie den Export der Ergebnisse im JSON-Format.

In [3]:
#Import
import os
import re
import json
import pandas as pd

### 1.2 Einlesen der OpenJur-Urteilstexte 

In diesem Schritt werden alle identifizierten Urteilstexte aus dem Datenverzeichnis eingelesen. Jede Datei wird über den Dateinamen einer eindeutigen Fallkennung (`case_id`) zugeordnet. Die Texte bilden die Rohdatenbasis für die nachfolgenden Extraktions- und Filterprozesse. Der Datenpfad wird im Code parametriert (`DATA_DIR`), um eine reproduzierbare Ausführung zu gewährleisten.

In [4]:
# (.txt) Dateien einlesen
DATA_DIR = "../data/Gerichtsurteile_Openjur" 
files = [f for f in os.listdir(DATA_DIR) if f.lower().endswith(".txt")]

print("Pfad:", os.path.abspath(DATA_DIR))
print("Anzahl .txt:", len(files))
print("Erste 10 Dateien:", files[:10])


Pfad: c:\Users\humme\OneDrive\Dokumente\Uni Ulm\ds_law\backend\data\Gerichtsurteile_Openjur
Anzahl .txt: 2375
Erste 10 Dateien: ['2090187.txt', '2112111.txt', '2112115.txt', '2112117.txt', '2112118.txt', '2112119.txt', '2112121.txt', '2112123.txt', '2124977.txt', '2126821.txt']


---

## 2. Extraktion relevanter Urteilsbestandteile und Selektion der Landgerichtsurteile

In diesem Abschnitt werden die eingelesenen Urteilstexte weiterverarbeitet, um für die nachfolgende Analyse relevante Textbestandteile gezielt zu extrahieren. Hierzu zählen insbesondere ein begrenzter Kopfbereich zur Voranalyse sowie der Tenor als Kern der gerichtlichen Entscheidung. Die strukturierte Aufbereitung dieser Textsegmente bildet die Grundlage für Filter-, Klassifikations- und Extraktionsschritte in den folgenden Abschnitten.

### 2.1 Aufbau des DataFrames und Extraktion eines Kopfbereichs

Die eingelesenen Texte werden in einem DataFrame (`df`) gespeichert. Zusätzlich wird ein begrenzter Kopfbereich (`head`) aus den ersten Zeichen extrahiert, da strukturelle Metadaten wie Gerichtstyp, Entscheidungsart und Zitierzeilen typischerweise am Anfang des Dokuments auftreten. Dieser Kopfbereich dient als effizienter Suchraum für die spätere Identifikation von Landgerichtsurteilen.


In [5]:
rows = []
for fn in files:
    case_id = fn.replace(".txt", "")
    path = os.path.join(DATA_DIR, fn)
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    rows.append({"case_id": case_id, "text": text})

df = pd.DataFrame(rows)
print("Gesamt eingelesen:", len(df))


Gesamt eingelesen: 2375


In [6]:
HEAD_CHARS = 8000
df["head"] = df["text"].astype(str).str.slice(0, HEAD_CHARS)

print("Head-Länge (Beispiel):", len(df.loc[0, "head"]))


Head-Länge (Beispiel): 8000


### 2.2 Extraktion des Tenors

Der Tenor enthält die eigentliche gerichtliche Entscheidung und ist daher für die inhaltliche Bewertung besonders relevant. Mithilfe regulärer Ausdrücke wird der Textabschnitt zwischen der Überschrift „Tenor“ und den nachfolgenden Abschnitten (z. B. „Tatbestand“ oder „Gründe“) extrahiert und in einer separaten Spalte gespeichert.

In [28]:
def extract_tenor(text: str) -> str:
    if not isinstance(text, str):
        return ""

    m_start = re.search(r"\bTenor\b", text, flags=re.IGNORECASE)
    if not m_start:
        return ""

    start = m_start.end()

    # Begrenztes Suchfenster nach dem Tenor (robuster gegen Navigation)
    window = text[start:start + 20000]

    m_end = re.search(
        r"\b(Tatbestand|Gründe|Gruende|Entscheidungsgründe|Entscheidungsgruende)\b",
        window,
        flags=re.IGNORECASE
    )

    end = start + m_end.start() if m_end else min(len(text), start + 8000)
    return text[start:end].strip()
df["tenor"] = df["text"].apply(extract_tenor)
print("Tenor vorhanden:", (df["tenor"].str.len() > 0).sum(), "von", len(df))


Tenor vorhanden: 2362 von 2375


### 2.3 Identifikation von Landgerichtsurteilen (LG)

Die Selektion der Landgerichtsurteile erfolgt anhand einer OpenJur-spezifischen Zitierzeile im Kopfbereich (Regex: „Einfach“ gefolgt von „LG“). Auf dieser Grundlage wird eine boolesche Variable erzeugt und der Teilkorpus df_lg gebildet.

In [8]:
# Wir suchen nach der Zeile, die mit "Einfach" beginnt, gefolgt von "LG"
# Der Regex r"Einfach\s*\n\s*LG" stellt sicher, dass LG direkt darunter steht
pattern_zitierung_lg = r"Einfach\s*\n\s*LG"

# Wir wenden das auf die Spalte an, die den Kopftext enthält
df["is_landgericht"] = df["head"].str.contains(pattern_zitierung_lg, regex=True, na=False)

# Jetzt erstellen wir den sauberen Dataframe
df_lg = df[df["is_landgericht"] == True].copy()

print("-" * 40)
print(f"✅ Echte LG-Urteile (über Zitierzeile): {len(df_lg)}")
print("-" * 40)

----------------------------------------
✅ Echte LG-Urteile (über Zitierzeile): 1189
----------------------------------------


### 2.4 Segmentierung der Urteile in juristische Abschnitte
Für die spätere Extraktion werden die Urteile in juristisch sinnvolle Teile zerlegt: Rubrum, Tenor, Tatbestand und Entscheidungsgründe. Dadurch kann das Modell gezielt relevante Passagen verarbeiten.
Die Segmentierung dient dazu, spätere Analysen gezielt auf entscheidungsrelevante Abschnitte (insb. Tenor und Entscheidungsgründe) zu fokussieren.


In [9]:
def split_judgment(text):
    """
    Teilt ein Urteil in Rubrum, Tenor, Tatbestand und Entscheidungsgründe auf.
    """
    segments = {
        "rubrum": "",
        "tenor": "",
        "tatbestand": "",
        "entscheidungsgruende": ""
    }
    
    # Muster für die Abschnittsüberschriften
    # Das Rubrum ist alles vor dem Tenor
    m_tenor = re.search(r"\bTenor\b", text, re.IGNORECASE)
    m_tatbestand = re.search(r"\bTatbestand\b", text, re.IGNORECASE)
    m_gruende = re.search(r"\b(Entscheidungsgründe|Gründe)\b", text, re.IGNORECASE)
    
    if m_tenor:
        segments["rubrum"] = text[:m_tenor.start()].strip()
        
        # Tenor bis Tatbestand
        if m_tatbestand:
            segments["tenor"] = text[m_tenor.end():m_tatbestand.start()].strip()
            
            # Tatbestand bis Gründe
            if m_gruende:
                segments["tatbestand"] = text[m_tatbestand.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tatbestand"] = text[m_tatbestand.end():].strip()
        else:
            # Falls kein Tatbestand gefunden wird, Tenor bis zum Ende oder Gründen
            if m_gruende:
                segments["tenor"] = text[m_tenor.end():m_gruende.start()].strip()
                segments["entscheidungsgruende"] = text[m_gruende.end():].strip()
            else:
                segments["tenor"] = text[m_tenor.end():].strip()
                
    return segments

# Beispielanwendung auf den Dataframe
df_lg['segments'] = df_lg['text'].apply(split_judgment)

## 3 Prompt-Generierung und Pilotierung der LLM-Extraktion (Gemini Batch)
Um API- und Token-Limits zu berücksichtigen, werden OpenJur-spezifische Navigationselemente aus dem Rubrum entfernt und alle Abschnitte in ihrer Länge begrenzt. Auf Basis dieser vorverarbeiteten Textsegmente wird ein standardisierter Prompt generiert, der die Extraktion der abgestimmten Variablen im JSON-Format steuert.
Die segmentweise Längenbegrenzung dient der Einhaltung von Token-Limits sowie der Reduktion von Kosten und Laufzeit, ohne entscheidungsrelevante Passagen (insb. Tenor und Entscheidungsgründe) zu verlieren.


### 3.1 Aufbereitung der Segmente und Definition des Extraktions-Prompts

In [29]:
def clean_rubrum(rubrum: str) -> str:
    if not isinstance(rubrum, str):
        return ""

    blacklist = [
        "rechtsprechung", "aktuell", "trending", "filter",
        "über openjur", "spenden", "api", "hilfe",
        "startseite", "bundesland", "gerichtsbarkeit",
        "impressum", "datenschutz", "nutzungsbedingungen",
        "fachzeitschriften", "suchen", "changelog", "einfach",
        "json", "bibtex", "ris"
    ]

    lines = []
    for line in rubrum.splitlines():
        l = line.strip().lower()
        if not l:
            continue
        if any(b in l for b in blacklist):
            continue
        lines.append(line.strip())

    return "\n".join(lines[:5])   

def slim_segments(segments):
    return {
        "rubrum": clean_rubrum(segments.get("rubrum") or "")[:2500],
        "tenor": (segments.get("tenor") or "")[:4000],
        "tatbestand": (segments.get("tatbestand") or "")[:3500],
        "entscheidungsgruende": (segments.get("entscheidungsgruende") or "")[:7000],
    }

def get_gemini_prompt(segments):
    """
    Erstellt den finalen Prompt basierend auf den Urteilssegmenten.
    """
    s = slim_segments(segments)   # <--- DAS ist der entscheidende Schritt

    prompt = f"""
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
{s['rubrum']}

TENOR (Ergebnis): 
{s['tenor']}

TATBESTAND (Sachverhalt): 
{s['tatbestand']}

ENTSCHEIDUNGSGRÜNDE (Rechtliche Würdigung): 
{s['entscheidungsgruende']}

### EXTRAKTIONS-AUFGABE:
Extrahiere folgende Variablen (bei Nichtfinden 'null' angeben):

1. **Input-Variablen (Features):**
   - Dieselmotor_Typ
   - Art_Abschalteinrichtung
   - KBA_Rueckruf
   - Fahrzeugstatus
   - Fahrzeugmodell_Baureihe
   - Update_Status
   - Kilometerstand_Kauf
   - Kilometerstand_Klageerhebung
   - Erwartete_Gesamtlaufleistung
   - Kaufdatum
   - Uebergabedatum
   - Datum_Klageerhebung
   - Nachweis_Aufklaerung
   - Beklagten_Typ
   - Datum_Urteil
   - Kaufpreis
   - Nacherfuellungsverlangen_Fristsetzung
   - Klageziel
   - Rechtsgrundlage

2. **Zielvariablen (Labels):**
   - LABEL_Anspruch_Schadensersatz
   - LABEL_Schadensersatzhoehe_Betrag
   - LABEL_Schadensersatzhoehe_Range

### AUSGABEFORMAT:
Antworte NUR mit einem validen JSON-Objekt in einer Liste:
[{{
  "case_id": "...",
  "Dieselmotor_Typ": null,
  "Art_Abschalteinrichtung": null,
  "KBA_Rueckruf": null,
  "Fahrzeugstatus": null,
  "Fahrzeugmodell_Baureihe": null,
  "Update_Status": null,
  "Kilometerstand_Kauf": null,
  "Kilometerstand_Klageerhebung": null,
  "Erwartete_Gesamtlaufleistung": null,
  "Kaufdatum": null,
  "Uebergabedatum": null,
  "Datum_Klageerhebung": null,
  "Nachweis_Aufklaerung": null,
  "Beklagten_Typ": null,
  "Datum_Urteil": null,
  "Kaufpreis": null,
  "Nacherfuellungsverlangen_Fristsetzung": null,
  "Klageziel": null,
  "Rechtsgrundlage": null,
  "LABEL_Anspruch_Schadensersatz": null,
  "LABEL_Schadensersatzhoehe_Betrag": null,
  "LABEL_Schadensersatzhoehe_Range": null
}}]
""".strip()
    return prompt


### 3.2 Erstellung eines Pilot-Inputs im JSONL-Format
Zur technischen Validierung der Analysepipeline wird ein Pilotdatensatz erzeugt, der eine begrenzte Anzahl von Landgerichtsurteilen umfasst. Für jedes ausgewählte Urteil werden die zuvor definierten Textsegmente extrahiert, zu einem standardisierten Analyse-Prompt zusammengeführt und im JSONL-Format gespeichert. Diese Pilotdatei dient als Testeingabe für die nachgelagerte Verarbeitung über die Gemini-API, bevor eine Skalierung auf den vollständigen Datensatz erfolgt.
Der Pilot dient ausschließlich der technischen Validierung der Prompt-Struktur und der Batch-Pipeline und ist nicht für eine inhaltliche Evaluation der Extraktionsergebnisse vorgesehen.

In [30]:
PILOT_N = 10
pilot_path = "gemini_batch_input_pilot_10.jsonl"

with open(pilot_path, "w", encoding="utf-8") as f:
    for _, row in df_lg.head(PILOT_N).iterrows():
        segments = row["segments"]
        full_prompt = get_gemini_prompt(segments)

        payload = {
            "custom_id": f"case_{row['case_id']}",
            "contents": [{
                "role": "user",
                "parts": [{"text": full_prompt}]
            }]
        }
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

print("✅ Pilot erstellt:", pilot_path)


✅ Pilot erstellt: gemini_batch_input_pilot_10.jsonl


Der Code dient der inhaltlichen und technischen Validierung der erzeugten Pilotdatei. Hierzu wird der erste Eintrag der JSONL-Datei geladen und exemplarisch ausgegeben, um Struktur, Inhalt und Länge des generierten Analyse-Prompts zu überprüfen

In [12]:
with open("gemini_batch_input_pilot_10.jsonl", "r", encoding="utf-8") as f:
    first = json.loads(f.readline())

print(first["custom_id"])
print(first["contents"][0]["parts"][0]["text"][:800])
print("Prompt-Länge:", len(first["contents"][0]["parts"][0]["text"]))


case_2090187
Analysiere die folgenden Abschnitte eines Gerichtsurteils zum Dieselskandal und extrahiere die Variablen präzise als JSON-Liste. 

### URTEILS-BESTANDTEILE:
RUBRUM (Kopfbereich mit Gericht & Datum): 
Rechtsgebiet
Gericht
Informationen

TENOR (Ergebnis): 
I. Die Klage wird abgewiesen.II. Der Kläger hat die Kosten des Rechtsstreits zu tragen.III. Das Urteil ist gegen Sicherheitsleistung in Höhe des 1,1-fachen des zu vollstreckenden Betrags vorläufig vollstreckbar.IV. Der Streitwert wird auf 31.234,00 € festgesetzt.

TATBESTAND (Sachverhalt): 
Der Kläger begehrt Lieferung eines mangelfreien Pkw.Der Kläger erwarb von der Beklagten im Jahr 2014 einen Neuwagen VW Passat 2,0 l TDI für 31.234,00 €. Der Pkw ist von dem "VW-Abgasskandal" betroffen. Der Kläger hat die Beklagte im Jahr 2016 durch Anwa
Prompt-Länge: 8908


### 3.3 Upload und Start eines Pilot-Batch-Jobs
In diesem Schritt wird die zuvor erzeugte Pilot-JSONL-Datei als Eingabe für die Gemini-API hochgeladen. Die Datei enthält strukturierte Analyseanfragen für mehrere Urteile und wird auf den Servern bereitgestellt, sodass sie anschließend im Rahmen einer Batch- oder sequenziellen Verarbeitung vom Sprachmodell verarbeitet werden kann. Der Upload erzeugt eine referenzierbare Eingabedatei, die anschließend einem eindeutig benannten Batch-Job zugewiesen wird und damit eine reproduzierbare Verarbeitung durch das Sprachmodell ermöglicht.


Initialisierung des API-Clients

In [31]:
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise RuntimeError("GEMINI_API_KEY ist nicht gesetzt")

client = genai.Client(api_key=api_key)
print("Client initialisiert")

Client initialisiert


Upload der JSONL-Datei

In [14]:
uploaded = client.files.upload(
    file="gemini_batch_input_pilot_10.jsonl",
    config={
        "display_name": "diesel-lg-pilot-10",
        "mime_type": "text/plain"
    }
)
print("Upload:", uploaded.name)


Upload: files/8i63btgg96qz


In [ ]:
job = client.batches.create(
    model="models/gemini-2.5-flash",
    src=uploaded.name,
    config={"display_name": "diesel-lg-pilot-10"}
)

print("Batch gestartet:", job.name)


## 4 Verarbeitung der Modellantworten und Erstellung des Extraktions-Datensatzes

In diesem Abschnitt werden die Batch-Ausgaben der Gemini-API eingelesen, validiert und in ein tabellarisches Format überführt. Zum Zeitpunkt der aktuellen Notebook-Version liegt lediglich der Pilot-Workflow vor; der Code zur Verarbeitung des vollständigen Batch-Outputs wird nach Abschluss des Batch-Jobs ergänzt.

(gemini_batch_input_NUR_LG.jsonl)

In [ ]:
# Hier Code einfügen

### 4.1 Download/Export der Batch-Ausgabedatei (JSONL) (Platzhalter)

Nach der in Abschnitt 2 beschriebenen Aufbereitung der Urteilstexte liegt der vollständige Analyse-Datensatz in Form einer strukturierten JSONL-Datei vor. Diese Datei dient in diesem Schritt als Eingabe für die automatisierte Verarbeitung durch ein großes Sprachmodell.

Die JSONL-Datei wird zunächst in das Batch-System hochgeladen. Anschließend wird ein Batch-Verarbeitungsjob gestartet, der die hochgeladene Datei als Eingabequelle verwendet. Für jedes enthaltene Dokument erzeugt das Modell eine strukturierte Antwort gemäß den im Prompt definierten Extraktionsvorgaben.

Als Ergebnis des Batch-Jobs stellt die API eine Ausgabedatei bereit, die die Modellantworten zu allen verarbeiteten Urteilen enthält. Diese Ausgabedatei liegt ebenfalls im JSONL-Format vor und bildet die Grundlage für die weitere Aufbereitung und Auswertung der Ergebnisse.


In [ ]:
# Hier Code einfügen

### 4.2 Parsing, Validierung und Tabellierung der Extraktionen (Platzhalter)

Die im vorherigen Schritt erzeugte Ausgabedatei des Batch-Jobs liegt zunächst als Rohdaten im JSONL-Format vor. Jede Zeile dieser Datei enthält die strukturierte Modellantwort zu einem einzelnen Landgerichtsurteil.

Diese Rohdaten werden lokal gespeichert und anschließend in ein tabellarisches Format überführt. Hierzu werden die relevanten Felder aus den JSON-Strukturen extrahiert und in einer einheitlichen Datenstruktur zusammengeführt, beispielsweise in Form einer CSV-Datei. 
Der so erzeugte Datensatz bildet die Grundlage für die weitere statistische Auswertung und Analyse in den folgenden Abschnitten.

In [ ]:
# Hier Code einfügen

### 4.3 Zusammenführung mit Metadaten und Speicherung (CSV/Parquet) (Platzhalter)

---

## 5. Datenaufbereitung für maschinelles Lernen

In diesem Abschnitt werden die Urteilstexte für die nachgelagerte prädiktive Modellierung aufbereitet. Hierzu erfolgt zunächst eine juristisch angepasste Textvorverarbeitung und die Ableitung numerischer Textrepräsentationen. Die für die supervised Lernphase erforderlichen Zielvariablen werden im Rahmen der LLM-basierten Extraktion (Abschnitt 4) erzeugt und anschließend mit den Textmerkmalen zusammengeführt (Abschnitt 5.4).
Ziel der Datenaufbereitung ist es, die extrahierten Merkmale in eine konsistente, auswertbare Form zu überführen, fehlende oder uneinheitliche Angaben zu behandeln und die Zielvariablen für die spätere Analyse eindeutig zu definieren.

### 5.1 Juristische Textvorverarbeitung

In [32]:
import pandas as pd
import re
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Setup: Spezialisiertes deutsches Sprachmodell laden
try:
    nlp = spacy.load("de_core_news_lg", disable=["ner", "parser"])
except Exception:
    print("Bitte installiere das spacy Modell: python -m spacy download de_core_news_lg")

# --- 2. JURISTISCHE TEXTVORVERARBEITUNG ---
def legal_preprocess(text):
    """
    Bereitet juristische Texte auf, indem Rauschen entfernt wird, 
    während rechtlich relevante Zahlen und Kontexte geschützt werden.
    """
    if not isinstance(text, str) or not text:
        return ""

    # NEU: START DES URTEILS FINDEN (Rauschschnitt Anfang) ---
    # Wir schneiden Webseiten-Menüs ("trending", "suche" etc.) weg
    start_keywords = ["tenor", "entscheidungsgründe", "tatbestand", "urteil", "beschluss", "endurteil"]
    text_lower_start = text.lower()
    
    # Finde die früheste Position eines der Keywords
    found_positions = [text_lower_start.find(kw) for kw in start_keywords if text_lower_start.find(kw) != -1]
    if found_positions:
        text = text[min(found_positions):]

    # NEU: ENDE DES URTEILS FINDEN (Rauschschnitt Ende) ---
    # Wir schneiden Impressum und Footer weg
    end_keywords = ["impressum", "nutzungsbedingungen", "nach oben", "datenschutz"]
    text_lower_end = text.lower()
    for ekw in end_keywords:
        e_pos = text_lower_end.find(ekw)
        if e_pos != -1:
            text = text[:e_pos]
            break

    # 1. Bereinigung von Rauschen (HTML-Tags, Sonderzeichen)
    text = re.sub(r'<.*?>', ' ', text)

    # 2. Schutz von Zahlen & Paragraphen (Platzhalter statt Löschen)
    # Euro-Beträge schützen
    text = re.sub(r'\d{1,3}(?:\.\d{3})*(?:,\d+)?\s*(?:EUR|€|Euro)', ' PLATZHALTER_BETRAG ', text)
    # Paragraphen schützen
    text = re.sub(r'§+\s*\d+[a-z]?\s*(?:\w+)?', ' PLATZHALTER_PARAGRAPH ', text)
    # Jahreszahlen schützen
    text = re.sub(r'\b(19|20)\d{2}\b', ' PLATZHALTER_JAHR ', text)

    # 3. Kleinschreibung zur Reduktion der Varianz
    text = text.lower()

    # 4. Tokenisierung und Lemmatisierung mit SpaCy
    doc = nlp(text)
    
    # 5. Kontextsensitive Stoppwort-Entfernung
    # Wichtige juristische Negationen schützen
    protected_negations = {"nicht", "kein", "ohne", "gegen", "trotz"}
    custom_stop_words = nlp.Defaults.stop_words - protected_negations
    
    # Extraktion der Lemmata (Grundformen)
    tokens = [
        token.lemma_ for token in doc 
        if token.lemma_ not in custom_stop_words 
        and not token.is_punct 
        and not token.is_space
        and len(token.text) > 1 # Token mit Länge 1 entfernen
    ]
    
    return " ".join(tokens)

# --- 2.5 HILFSFUNKTIONEN: Simulation + echtes Batch lesen ---
def get_llm_text(r: dict) -> str:
    # Simulation (simulated_batch_output.jsonl)
    if "text" in r:
        return r["text"]
    # Echtes Batch (später)
    if "response" in r:
        return r["response"]["body"]["choices"][0]["message"]["content"]
    raise KeyError("Unbekanntes Ergebnisformat (kein 'text' und kein 'response').")

def parse_llm_json(text: str) -> dict:
    # Entfernt ```json ... ``` falls vorhanden
    text = re.sub(r"^```json\s*|\s*```$", "", text.strip(), flags=re.MULTILINE)
    # Falls außenrum Text steht: ersten JSON-Block extrahieren
    m = re.search(r"(\{.*\})", text, flags=re.DOTALL)
    if m:
        text = m.group(1)
    return json.loads(text)

# --- 3. MERGING DER DATEN (URTEILE + EXTRAKTIONEN) ---
def merge_and_finalize(judgment_file, batch_results_file):
    """
    Führt die ursprünglichen Urteilstexte mit den Gemini-Extraktionen zusammen.
    """
    # 1. Laden der aufbereiteten LG-Urteile
    df_judgments = pd.read_json(judgment_file, lines=True)
    df_judgments['case_id'] = df_judgments['custom_id'].str.replace('case_', '')

    # 2. Laden der Gemini-Batch-Ergebnisse
    with open(batch_results_file, 'r', encoding='utf-8') as f:
        results = [json.loads(line) for line in f]
    
    extracted_rows = []
    for r in results:
        try:
            case_id = r['custom_id'].replace('case_', '')
            llm_text = get_llm_text(r)
            content = parse_llm_json(llm_text)
            content['case_id'] = case_id
            extracted_rows.append(content)
        except Exception:
            continue
            
    df_extracted = pd.DataFrame(extracted_rows)

    # 3. Zusammenführung über case_id 
    df_final = pd.merge(df_judgments, df_extracted, on='case_id', how='inner')

    # 4. Textverarbeitung anwenden
    print("Starte Textvorverarbeitung...")
    df_final['cleaned_text'] = df_final['text'].apply(legal_preprocess)

    return df_final

# --- 4. MODELL-VORBEREITUNG (TF-IDF) ---
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),   # Bigramme erhalten Wortzusammenhänge
    max_features=1000,    # Reduktion der Komplexität
    min_df=5              # Seltene Begriffe ignorieren
)

In [ ]:
#RESULTS_FILE = "simulated_batch_output.jsonl"   # später echte Batch-Output-Datei
# df_final = merge_and_finalize(
    judgment_file="lg_judgments.jsonl",
    batch_results_file=RESULTS_FILE
)

In [17]:
from tqdm import tqdm
tqdm.pandas()

# 1) Sicherstellen, dass Segmente existieren
if "segments" not in df_lg.columns:
    df_lg["segments"] = df_lg["text"].apply(split_judgment)

df_final = df_lg.copy()

# 2) Text für Embeddings: TENOR + ENTSCHEIDUNGSGRÜNDE
def build_text_for_embedding(s):
    if not isinstance(s, dict):
        return ""
    return (s.get("tenor") or "") + "\n" + (s.get("entscheidungsgruende") or "")

df_final["text_for_embedding"] = df_final["segments"].apply(build_text_for_embedding)

# 3) Länge begrenzen (wichtig für Laufzeit)
MAX_CHARS = 12000
df_final["text_for_embedding"] = (
    df_final["text_for_embedding"]
    .astype(str)
    .str.slice(0, MAX_CHARS)
)

# 4) Juristisches Preprocessing (einmal, mit Fortschritt)
df_final["cleaned_text"] = df_final["text_for_embedding"].apply(legal_preprocess)

# 5) Sanity-Checks
print("df_final shape:", df_final.shape)
print(
    "non-empty cleaned_text:",
    (df_final["cleaned_text"].str.len() > 0).sum()
)

# Vorschau
print("\nBeispiel cleaned_text:\n")
print(df_final["cleaned_text"].iloc[0][:300])


df_final shape: (1189, 8)
non-empty cleaned_text: 1185

Beispiel cleaned_text:

Urteil gegen Sicherheitsleistung Höhe 1,1-fache vollstreckend betrag vorläufig vollstreckbar.iv Streitwert Platzhalter_betrag festsetzen zulässig Klage unbegründet.d Kläger gegen beklagen Anspruch Nachlieferung mangelfrei Pkw aktuell Produktion Platzhalter_paragraph Platzhalter_paragraph Alternative


### 5.2 Text-Vektorisierung mittels TF-IDF
In diesem Schritt transformieren wir die bereinigten Urteilstexte mithilfe des TF-IDF-Verfahrens in ein numerisches Format, das für Machine-Learning-Algorithmen lesbar ist. Im Gegensatz zu abstrakten Embeddings bietet TF-IDF eine hohe Interpretierbarkeit, da jedes Merkmal einem konkreten juristischen Begriff oder einer Wortkombination (N-Gramm) entspricht. Durch die Begrenzung auf die 1.000 relevantesten Begriffe reduzieren wir das Rauschen im Datensatz und bereiten die Daten optimal auf den in der Aufgabenstellung empfohlenen Entscheidungsbaum vor.

In [38]:
# Semantische Repräsentation mittels TF-IDF (statt Word2Vec)
from sklearn.feature_extraction.text import TfidfVectorizer

# Wir nehmen Unigramme und Bigramme, um Begriffe wie "Klage abgewiesen" zu erfassen.
tfidf = TfidfVectorizer(
    ngram_range=(1, 2), 
    max_features=1000,  # Reduziert die Komplexität für den Baum
    min_df=5,           # Wort muss in mind. 5 Urteilen vorkommen
    stop_words=None      # Stoppwörter wurden bereits im Preprocessing entfernt
)

# Erstellt die Feature-Matrix
X_tfidf = tfidf.fit_transform(df_final["cleaned_text"].fillna(""))
feature_names = tfidf.get_feature_names_out()

print(f"Feature-Matrix erstellt: {X_tfidf.shape[0]} Urteile, {X_tfidf.shape[1]} Begriffe.")

Feature-Matrix erstellt: 1189 Urteile, 1000 Begriffe.


In [ ]:
# Tokenisierte Texte für Word2Vec
#sentences = [str(t).split() for t in df_final["cleaned_text"].dropna()]
# Skip-Gram Word2Vec Modell trainieren
#from gensim.models import Word2Vec

#w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=200,   # Dimension der Wortvektoren
    window=8,          # Kontextfenster
    min_count=5,       # sehr seltene Wörter ignorieren
    workers=4,
    sg=1,              # <-- Skip-Gram (besser für Fachbegriffe, prüfen) 
    epochs=10
)

In [ ]:
# Dokumenten-Vektor durch Mittelung der Wortvektoren
#import numpy as np

#def document_vector(doc, model):
    if not isinstance(doc, str) or not doc.strip():
        return np.zeros(model.vector_size)
    words = doc.split()
    vectors = [model.wv[w] for w in words if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Alle Urteile vektorisieren
X_embeddings = np.vstack(
    df_final["cleaned_text"].apply(lambda x: document_vector(x, w2v_model))
)


### 5.3 Aufbau des Analyse-Datensatzes
Nach der Vektorisierung führen wir die mathematischen Ergebnisse in einer strukturierten Feature-Matrix zusammen. Wir wandeln die Sparse-Matrix in einen übersichtlichen DataFrame um und verknüpfen jedes Urteil über die eindeutige case_id mit seinen Textmerkmalen. Diese Struktur ist essentiell, um im nächsten Schritt die durch das LLM extrahierten Zielvariablen (Schadensersatz oder Abweisung) präzise jeder Beobachtung zuordnen zu können. Damit stellen wir sicher, dass der Datensatz modellunabhängig konzipiert ist und eine solide Basis für die nachgelagerte prädiktive Modellierung bietet.

In [39]:
# Aufbau des Analyse-Datensatzes
df_features = pd.DataFrame(X_tfidf.toarray(), columns=feature_names)
df_features.insert(0, "case_id", df_final["case_id"].reset_index(drop=True).values)

print("Analyse-Datensatz bereit für Merge mit Labels.")

Analyse-Datensatz bereit für Merge mit Labels.


In [41]:
# Aufbau des Analyse-Datensatzes

# Feature-Namen für die Embeddings
#emb_cols = [f"emb_{i}" for i in range(X_embeddings.shape[1])]

# Embeddings als DataFrame
#df_features = pd.DataFrame(X_embeddings, columns=emb_cols)

# case_id ergänzen (für spätere Joins mit Labels)
#df_features.insert(0, "case_id", df_final["case_id"].reset_index(drop=True).values)

# optional: Duplikate prüfen (sollte 0 sein)
#print("Duplicate case_id:", df_features["case_id"].duplicated().sum())

#print("df_features shape:", df_features.shape)
#df_features[["case_id"] + [c for c in df_features.columns if c.startswith("emb_")]].head()




Der Analyse-Datensatz besteht aus 1.189 Beobachtungen (Urteilen) mit jeweils 200 numerischen Merkmalen, die den semantischen Gehalt der Entscheidungsgründe abbilden

### 5.4 Modellierung und Evaluation (nach Verfügbarkeit der Labels)
Auf Grundlage des in Abschnitt 5.3 aufgebauten Analyse-Datensatzes erfolgt im Folgenden die prädiktive Modellierung. Hierzu werden die semantischen Dokumenten-Embeddings mit den aus der automatisierten Extraktion gewonnenen Zielvariablen verknüpft und für den Einsatz überwachter Lernverfahren vorbereitet.

In [ ]:
# --- 5.4 (wird aktiviert sobald df_labels aus Batch da ist) ---

# df_ml = df_features.merge(df_labels, on="case_id", how="inner")
# X = df_ml.filter(like="emb_").values
# y = df_ml["LABEL_Anspruch_Schadensersatz"].astype(int).values

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42, stratify=y
# )

# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1, class_weight="balanced")
# rf.fit(X_train, y_train)

# from sklearn.metrics import classification_report, confusion_matrix
# y_pred = rf.predict(X_test)
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))


## 6. Analyse und Auswertung